# Time Series Logistic Regression Algorithm

This algorithm is used to forecast FOREX prices for the USDJPY pair. It fetches data from MetaTrader5 and applies feature engineering to create additional data.

The forecasted values are set to:
- **Position 1** if the prediction indicates that the price will go up in the next hour
- **Position 0** if the prediction indicates that the price will go down in the nex hourx hour

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-darkgrid")
from matplotlib import cycler
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import MetaTrader5 as mt5
from datetime import datetime
from UsefulFunctions import data, backtest
import ta

In [68]:
# Initiate bound between MetaTrader5 and Python
mt5.initialize()

True

In [70]:
# Fetching data from MetaTrader5 from 2015 to the present using the custom function "get_rates" from the data.py file
# The historical data retrieved is of 1-hour timeframe
df = data.get_rates("USDJPY", mt5.TIMEFRAME_H1, datetime(2015,1,1))
df.head()

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2015-01-02 08:00:00,120.42,120.44,120.33,120.39,262,30,0
2015-01-02 09:00:00,120.40,120.50,120.38,120.43,599,30,0
2015-01-02 10:00:00,120.44,120.48,120.34,120.34,832,30,0
2015-01-02 11:00:00,120.33,120.42,120.31,120.40,761,30,0
2015-01-02 12:00:00,120.41,120.47,120.37,120.43,484,30,0


In [72]:
# Drop "spread" and "real_volume" columns and remane columns "tick_volumne" to only "volumne"
df = df[["open", "high", "low", "close", "tick_volume"]]
df.rename(columns={'tick_volume': 'volume'}, inplace=True)
df.head()

,open,high,low,close,volume
time,,,,,
2015-01-02 08:00:00,120.42,120.44,120.33,120.39,262
2015-01-02 09:00:00,120.40,120.50,120.38,120.43,599
2015-01-02 10:00:00,120.44,120.48,120.34,120.34,832
2015-01-02 11:00:00,120.33,120.42,120.31,120.40,761
2015-01-02 12:00:00,120.41,120.47,120.37,120.43,484


## Feature Engineering 
**Creating new input variables from raw data**

In [75]:
# Create a second DataFrame from the close column and compute the percent change in a new column
df_copy = df[["close"]]
df_copy["pct_change"] = df_copy["close"].pct_change(1)
df_copy

,close,pct_change
time,,
2015-01-02 08:00:00,120.390,NaN
2015-01-02 09:00:00,120.430,0.000332
2015-01-02 10:00:00,120.340,-0.000747
2015-01-02 11:00:00,120.400,0.000499
2015-01-02 12:00:00,120.430,0.000249
...,...,...
2024-08-22 23:00:00,146.286,-0.000007
2024-08-23 00:00:00,146.215,-0.000485
2024-08-23 01:00:00,146.170,-0.000308


In [77]:
# New DataFrame with all indications from "ta" library and shifting them by one row
df_indicators = ta.add_all_ta_features(df, open="open", high="high", low="low", close="close", volume="volume", fillna=True).shift(1)
df_indicators

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02 09:00:00,120.420,120.440,120.330,120.390,262.0,2.381818e+01,262.0,0.090909,0.000000,0.000000,0.000000,0.000000,120.386667,50.000000,1000.000000,120.390000,120.390000,120.390000,0.000000,0.000000,0.0,0.0,120.386667,120.496667,120.276667,0.182744,0.515152,0.0,0.0,120.330,120.440,120.3850,0.091370,0.545455,0.000000,0.000000,0.000000,0.000000,0.000000,120.390000,120.390000,120.390000,120.390000,0.000000,0.000000,0.000000,1.122635,1.000000,-1.327603,11.150479,11.150479,0.000000,120.3850,120.385,120.38500,120.385,0.000000,0.000000,0.000000,0.000000,0.000000,119.040587,119.014357,0.0,0.0,0.0,120.310000,120.440000,0.0,0.0,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,54.545455,54.545455,-45.454545,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,120.390000,0.000000,0.000000,0.000000
2015-01-02 10:00:00,120.400,120.500,120.380,120.430,599.0,-7.601515e+01,861.0,-0.088287,23.960000,1101.836394,1101.836394,0.199020,120.421452,100.000000,1000.000000,120.410000,120.450000,120.370000,0.066440,0.750000,0.0,0.0,120.411667,120.526667,120.296667,0.191011,0.579710,0.0,0.0,120.330,120.500,120.4150,0.141184,0.588235,0.000000,0.000000,0.003191,0.000638,0.002553,120.410000,120.410000,120.396154,120.392963,0.113515,0.040064,0.073451,0.000065,2.014493,-1.347603,11.318458,11.234469,0.083990,120.4150,120.415,120.41500,120.415,0.000000,0.000000,0.000000,0.000000,66.666667,119.040587,119.014357,4.0,0.0,4.0,120.310000,120.440000,0.0,0.0,100.000000,0.000000,0.000000,0.000000,100.000000,21.739130,58.823529,56.684492,-41.176471,0.000000,0.000000,0.002650,0.000530,0.002120,9.368174,1.873635,7.494540,120.407998,0.033225,0.033220,0.033225
2015-01-02 11:00:00,120.440,120.480,120.340,120.340,832.0,-9.080152e+02,29.0,-0.536335,9.840000,-504.807692,298.514351,-0.422752,120.404357,41.868946,1000.000000,120.386667,120.460302,120.313031,0.122332,0.183125,0.0,0.0,120.403333,120.526667,120.280000,0.204867,0.243243,0.0,0.0,120.330,120.500,120.4150,0.141212,0.058824,0.000000,0.000000,-0.001525,0.000206,-0.001731,120.386667,120.386667,120.387515,120.389040,0.164875,0.134343,0.030532,0.000024,3.065995,-1.324269,11.122482,11.197140,-0.074657,120.4150,120.41

In [78]:
# Identifying columns with all zero values to avoid interference during model training
zero_columns_list = df_indicators.columns[(df_indicators == 0).all(axis=0)].tolist()
zero_columns_list

[]

In [79]:
# Concatenating both dataframes and drop the close column (which wasn't shifted) from df_copy
new_df = pd.concat((df_indicators, df_copy["pct_change"]), axis=1)
new_df.dropna(inplace=True)

# Making sure all columns are shown
pd.set_option('display.max_columns', None)

new_df.head()

,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,pct_change
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-02 09:00:00,120.42,120.44,120.33,120.39,262.0,23.818182,262.0,0.090909,0.000000,0.000000,0.000000,0.000000,120.386667,50.000000,1000.000000,120.390000,120.390000,120.390000,0.000000,0.000000,0.0,0.0,120.386667,120.496667,120.276667,0.182744,0.515152,0.0,0.0,120.33,120.44,120.385,0.091370,0.545455,0.0,0.0,0.000000,0.000000,0.000000,120.390000,120.390000,120.390000,120.390000,0.000000,0.000000,0.000000,1.122635,1.000000,-1.327603,11.150479,11.150479,0.000000,120.385,120.385,120.385,120.385,0.0,0.0,0.0,0.0,0.000000,119.040587,119.014357,0.0,0.0,0.0,120.31,120.4400,0.0,0.0,100.000000,0.0,0.0,0.0,0.000000,0.000000,54.545455,54.545455,-45.454545,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,120.390000,0.000000,0.000000,0.000000,0.000332
2015-01-02 10:00:00,120.40,120.50,120.38,120.43,599.0,-76.015152,861.0,-0.088287,23.960000,1101.836394,1101.836394,0.199020,120.421452,100.000000,1000.000000,120.410000,120.450000,120.370000,0.066440,0.750000,0.0,0.0,120.411667,120.526667,120.296667,0.191011,0.579710,0.0,0.0,120.33,120.50,120.415,0.141184,0.588235,0.0,0.0,0.003191,0.000638,0.002553,120.410000,120.410000,120.396154,120.392963,0.113515,0.040064,0.073451,0.000065,2.014493,-1.347603,11.318458,11.234469,0.083990,120.415,120.415,120.415,120.415,0.0,0.0,0.0,0.0,66.666667,119.040587,119.014357,4.0,0.0,4.0,120.31,120.4400,0.0,0.0,100.000000,0.0,0.0,0.0,100.000000,21.739130,58.823529,56.684492,-41.176471,0.0,0.0,0.002650,0.000530,0.002120,9.368174,1.873635,7.494540,120.407998,0.033225,0.033220,0.033225,-0.000747
2015-01-02 11:00:00,120.44,120.48,120.34,120.34,832.0,-908.015152,29.0,-0.536335,9.840000,-504.807692,298.514351,-0.422752,120.404357,41.868946,1000.000000,120.386667,120.460302,120.313031,0.122332,0.183125,0.0,0.0,120.403333,120.526667,120.280000,0.204867,0.243243,0.0,0.0,120.33,120.50,120.415,0.141212,0.058824,0.0,0.0,-0.001525,0.000206,-0.001731,120.386667,120.386667,120.387515,120.389040,0.164875,0.134343,0.030532,0.000024,3.065995,-1.324269,11.122482,11.197140,-0.074657,120.415,120.415,120.415,120.415,0.0,0.0,0.0,0.0,-50.000000,119.040587,119.014357,4.0,0.0,4.0,120.31,120.4400,0.0,1.0,29.213483,0.0,0.0,0.0,95.121951,13.513514,5.882353,39.750446,-94.117647,0.0,0.0,-0.001267,0.000171,-0.001437,20.231822,5.545272,14.686550,120.377425,-0.074732,-0.074760,-0.041532,0.000499
2015-01-02 12:00:00,120.33,120.42,120.31,120.40,761.0,-423.742424,790.0,-0.172674,14.957143,-650.459921,-17.810406,-0.043327,120.395770,27.335678,1000.498587,120.390000,120.454807,120.325193,0.107662,0.577152,0.0,0.0,120.396667,12